In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch; print(torch.__version__)

1.8.1+cu101


In [3]:
!pip install scanpy
!pip install pillow --upgrade
!pip install python-igraph
!pip install graspologic
!pip install networkx
!pip install matplotlib
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-geometric
!pip install captum

Requirement already up-to-date: pillow in /usr/local/lib/python3.7/dist-packages (8.2.0)
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html


In [4]:
!cp -r drive/MyDrive/epiClassify .
%cd epiClassify

/content/epiClassify


In [5]:
from GNNProject.utils import *
from GNNProject.dataset import *
from GNNProject.model import *
from GNNProject.classifier import *
from captum import *
from captum.attr import IntegratedGradients
import scanpy
import pandas as pd
import igraph as ig
import numpy as np

In [6]:
import torch
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)
print(device)

cuda:0


In [7]:
# dataset params
n_obs_train = None
n_obs_test  = None 
random_seed = 1996
alphas      = 2
# NN params
epochs   = 20
lr       = 0.001
momentum = .9

In [8]:
# make a Dataset instance 
dataset = Dataset(tag='embryo3_5', input_dir='sma03_scPotter/pred', random_seed=random_seed)
dataset.load()
dataset.subsample(n_obs_train=n_obs_train, n_obs_test=n_obs_test)

n_features = dataset.X_train.shape[1]
n_classes  = len(set(dataset.y_train.tolist()))
# infer the graph for train and test
# dataset.create_graph(alphas=2, method='glasso_R', mode='cd')
adj = load_adj('sma03_scPotter/pred/embryo3_5')[0]
dataset.Ah_train = dataset.Ah_test = adj
# # dataloaders with true graphs
# train_true = dataset._dataloader('train',use_true_graph=True,batch_size=16)
# test_true  = dataset._dataloader('test',use_true_graph=True,batch_size=16)
# dataloaders with inferred graphs
train_inferred = dataset._dataloader('train',use_true_graph=False,batch_size=16)
test_inferred  = dataset._dataloader('test',use_true_graph=False,batch_size=16)

# plot_lowDim(dataset.X_train, dataset.y_train, title='UMAP of Train Dataset')
# plt.show()
# plot_lowDim(dataset.X_test, dataset.y_test, title='UMAP of Test Dataset')
# plt.show()

In [9]:
# clf_MLP = Classifier(
#         n_features=n_features,
#         n_classes=n_classes,
#         n_hidden_GNN=[],
#         n_hidden_FC=[],
#         dropout_FC=0.5,
#         classifier='MLP', 
#         lr=0.0001,
#         momentum=momentum,
#         log_dir="runs/MLP_true",
#         device = device) 
# # fit the classifier on train data
# clf_MLP.fit(train_inferred, epochs = 20, test_dataloader=None,verbose=True)
# # evaluate the trained classifier on train data
# _ = clf_MLP.eval(train_inferred, verbose=True)
# _ = clf_MLP.eval(test_inferred, verbose=True)
# imp_train_MLP = clf_MLP.interpret(train_inferred, n_features=n_features, n_classes=n_classes)
# imp_test_MLP  = clf_MLP.interpret(test_inferred, n_features=n_features, n_classes=n_classes)
# imp_MLP = imp_train_MLP + imp_test_MLP
# save_np_txt(imp_MLP, 'imp_MLP.txt', colnames=dataset.classes, rownames=dataset.features)

In [10]:
clf_TransformerConv = Classifier(
        n_features=n_features,
        n_classes=n_classes,
        n_hidden_GNN=[8],
        dropout_FC=0.2,
        dropout_GNN=0.3,
        classifier='TransformerConv', 
        lr=0.001,
        momentum=momentum,
        log_dir="runs/TransformerConv_true",
        device = device) 
# fit the classifier on train data
clf_TransformerConv.fit(train_inferred, epochs = 30, test_dataloader=None,verbose=True)
# evaluate the trained classifier on train data
_ = clf_TransformerConv.eval(train_inferred, verbose=True)
# _ = clf_TransformerConv.eval(test_inferred, verbose=True)
imp_train_TransformerConv = clf_TransformerConv.interpret(train_inferred, n_features=n_features, n_classes=n_classes)
# imp_test_TransformerConv  = clf_TransformerConv.interpret(test_inferred, n_features=n_features, n_classes=n_classes)
imp_TransformerConv = imp_train_TransformerConv 
# + imp_test_TransformerConv
save_np_txt(imp_TransformerConv, 'output/imp_embryo3_5.txt', colnames=dataset.classes, rownames=dataset.features)

[1] loss: 1.312
[4] loss: 0.959
[7] loss: 0.912
[10] loss: 0.880
[13] loss: 0.849
[16] loss: 0.831
[19] loss: 0.823
[22] loss: 0.815
[25] loss: 0.808
[28] loss: 0.794
Accuracy: 0.774
Confusion Matrix:
 [[1132    5   20    9    3   16   12    0  170   10   29   19   41   49
    12   13]
 [   5  546    0    5    5    0    6    0   33   44   47    6    7    2
     3   23]
 [  15    1  379    0    0    4    0    0    5    0    7    4    5   64
     0    0]
 [  10    5    1  344   10    0    3    0   15   11   14   30    0    1
     0    1]
 [   1    4    1   22  264    0    2    0   18   17    4   23   11    0
     0    0]
 [  15    0    7    0    0  251    1    0    4    2    5   27   11   13
     3    0]
 [   4    6    2    0    1    2  179    0   92    2   17    0    1    1
     1   25]
 [   0    0    1    0    1    0    0  147    0    0    0    7    9    9
     3    0]
 [  51    6    5    6    1    1   19    0 1168    4   36   10    5    5
     1   10]
 [  10   23    3    2   13    1  

In [11]:
save_np_txt(imp_TransformerConv, 'output/imp_embryo3_5.txt', colnames=dataset.classes, rownames=dataset.features)

In [12]:
out = []
with torch.no_grad():
    for batch in test_inferred:
        x, edge_index, label= batch.x.to(clf_TransformerConv.device), batch.edge_index.to(clf_TransformerConv.device), batch.y.to('cpu')
        outputs = clf_TransformerConv.net(x, edge_index)
        out.append(outputs.to('cpu').numpy().reshape(-1))
out = np.asarray(out)
save_np_txt(out, 'output/preds_embryo3_5.txt', colnames=dataset.classes)